In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
headers={"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_2) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.0.2 Safari/605.1.15"
        }

In [8]:
icd_chap_url=u'http://code.nhsa.gov.cn:8000/jbzd/public/toStdTcmTreeList.html?partcode=B'
icd_chapter=requests.get(icd_chap_url,headers=headers)

In [9]:
icd_chapter=icd_chapter.json()

In [6]:
icd_chapter[0]['tcmId']

'20545f20-7930-11e9-9611-8cec4bd010f3'

In [7]:
icd_chapter=pd.DataFrame(icd_chapter)
icd_chapter.to_csv('./CHS_TCM_DISEASE_CHAPTER.csv')

In [17]:
icd_detail_url_base=u'http://code.nhsa.gov.cn:8000/jbzd/public/dataTcmSerachDetail.html?tcmId='

In [18]:
icd_detail_url_full=icd_detail_url_base+icd_chapter[0]['tcmId']

In [19]:
icd_detail_url_full

'http://code.nhsa.gov.cn:8000/jbzd/public/dataTcmSerachDetail.html?tcmId=20545f20-7930-11e9-9611-8cec4bd010f3'

In [20]:
icd_detail=requests.get(icd_detail_url_full,headers=headers)

In [21]:
if icd_detail.status_code == 200:
    info = {}
    soup = BeautifulSoup(icd_detail.text)
    icd_code_lst=[d.text.strip() for d in soup.find_all('a')]
    icd_name_lst=[d.text.strip() for d in soup.find_all('span')]


In [25]:
CHS_ICD10=[]
for i in range(len(icd_chapter)):
    icd_detail_url_full=icd_detail_url_base+icd_chapter[i]['tcmId']
    icd_detail=requests.get(icd_detail_url_full,headers=headers)
    if icd_detail.status_code == 200:
        info = {}
        soup = BeautifulSoup(icd_detail.text)
        icd_code_lst=[d.text.strip() for d in soup.find_all('a')]
        icd_name_lst=[d.text.strip() for d in soup.find_all('span')]
        df_icd=pd.DataFrame({'tcm_code':icd_code_lst,'tcm_name':icd_name_lst})
        CHS_ICD10.append(df_icd)
    else:
        print(icd_detail_url_full)

In [26]:
CHS_ICD10_Table=pd.concat(CHS_ICD10,axis=0,ignore_index=True)

In [27]:
CHS_ICD10_Table.head(20)

,tcm_code,tcm_name
0,BNF,肺系病类
1,BNF010,咳嗽病
2,BNF011,外感咳嗽病
3,BNF012,内伤咳嗽病
4,BNF020,肺痿病
5,BNF030,肺痈病
6,BNF040,哮病
7,BNF050,喘病
8,BNF051,暴喘病
9,BNF060,肺胀病


In [28]:
len(CHS_ICD10_Table)

1300

In [29]:
CHS_ICD10_Table.to_csv('./LOOKUP_TABLES/CHS_TCM_DISEASE.csv',encoding='utf-8',index=False)